![sql](images/sql-logo.jpg)

In [1]:
import pandas as pd
import sqlite3
import pandasql

conn = sqlite3.connect('data/flights.db')
cur = conn.cursor()

# Objectives

- Use SQL aggregation functions with GROUP BY
- Use HAVING for group filtering
- Use SQL JOIN to combine tables using keys

# Aggregating Functions

>  A SQL **aggregating function** takes in many values and returns one value.

We have already seen some SQL aggregating functions like `COUNT()`. There are also others, like SUM(), AVG(), MIN(), and MAX().

## Example Simple Aggregations

In [2]:
# Max value for longitude
pd.read_sql('''
    
SELECT MAX(CAST(longitude as real)) AS max_longitude
FROM airports
    
''', conn)

,max_longitude
0,179.951


In [3]:
# Max value for id in table
pd.read_sql('''
SELECT 
    MAX(CAST(id AS integer))
FROM 
    airports
''', conn)

,MAX(CAST(id AS integer))
0,9541


In [4]:
# Effectively counts all the inactive airlines 
pd.read_sql('''
    SELECT 
        COUNT()
    FROM 
        airlines
    WHERE 
        active='N'
''', conn)

,COUNT()
0,4886


We can also give aliases to our aggregations:

In [5]:
# Effectively counts all the active airlines 
pd.read_sql('''
    SELECT 
        COUNT() AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

,number_of_active_airlines
0,1161


# Grouping in SQL

We can go deeper and use aggregation functions on _groups_ using the `GROUP BY` clause.

The `GROUP BY` clause will group one or more columns together with the same values as one group to perform aggregation functions on.

## Example `GROUP BY`  Statements

Let's say we want to know how many active and non-active airlines there are.

### Without `GROUP BY`

Let's first start with just seeing how many airlines there are:

In [6]:
df_results = pd.read_sql('''

SELECT COUNT() AS num_of_airlines
FROM airlines
''', conn)

df_results

,num_of_airlines
0,6048


One way for us to get the counts for each is to create two queries that will filter each kind of airline (active vs non-active) and count those values:

In [7]:
df_active = pd.read_sql('''
    SELECT 
        COUNT() AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

df_inactive = pd.read_sql('''
    SELECT 
        COUNT() AS number_of_inactive_airlines
    FROM 
        airlines
    WHERE 
        active='N'
''', conn)

display(df_active)
display(df_inactive)

,number_of_active_airlines
0,1161


,number_of_inactive_airlines
0,4886


This works but it's inefficient.

### With `GROUP BY`

Instead, we can tell the SQL server to do the work for us by grouping values we care about for us!

In [8]:
df_results = pd.read_sql('''

SELECT COUNT() as num_of_airlines, active
FROM airlines
GROUP BY active

''', conn)

df_results

,num_of_airlines,active
0,4886,N
1,1161,Y
2,1,n


This is great! And if you look closely, you can observe we have _three_ different groups instead of our expected two!

## Group Task

- Which countries have the highest numbers of active airlines? Return the top 10.

In [10]:
# Note that we have a 'country' field/column
pd.read_sql('''
    SELECT
        *
    FROM
        airlines
''', conn).head()

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


In [18]:
pd.read_sql("""

SELECT country, COUNT() AS num_airlines
FROM airlines
WHERE active = "Y"
GROUP BY country
ORDER BY num_airlines DESC
LIMIT 10

""", conn)

,country,num_airlines
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34
5,Australia,26
6,China,25
7,Spain,24
8,Brazil,23
9,France,22


In [19]:
pandas_way = pd.read_sql("""
SELECT *
FROM airlines


""", conn)

In [20]:
pandas_way

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N
...,...,...,...,...,...,...,...,...,...
6043,6043,19828,Vuela Cuba,Vuela Cuba,6C,6CC,None,Cuba,Y
6044,6044,19830,All Australia,All Australia,88,8K8,None,Australia,Y
6045,6045,19831,Fly Europa,None,ER,RWW,None,Spain,Y
6046,6046,19834,FlyPortugal,None,PO,FPT,FlyPortugal,Portugal,Y


In [21]:
pandas_way.loc[pandas_way['active'] == 'Y']['country'].value_counts()[:10]

United States     141
Russia             72
United Kingdom     40
Germany            37
Canada             34
Australia          26
China              25
Spain              24
Brazil             23
France             22
Name: country, dtype: int64

<details>
    <summary><b>Possible Solution</b></summary>

``` sql
    pd.read_sql('''
    SELECT 
        COUNT() AS num,
        country
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    ORDER BY 
        num DESC
    LIMIT 10
    ''', conn)```
</details>

> Note that the `GROUP BY` clause is considered _before_ the `ORDER BY` and `LIMIT` clauses, more on this later.

## Exercise: Grouping

- Run a query that will return the number of airports by time zone. Each row should have a number of airports and a time zone.

In [22]:
# Your code here
pd.read_sql("""
SELECT *
FROM airports
LIMIT 5
""", conn)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


In [29]:
pd.read_sql("""

SELECT timezone, COUNT() AS num_airports, AVG(CAST(altitude AS int)) AS average_alt
FROM airports
GROUP BY timezone
ORDER BY num_airports DESC


""", conn)

,timezone,num_airports,average_alt
0,America/New_York,628,471.812102
1,America/Chicago,373,804.284182
2,Europe/Berlin,319,596.109718
3,America/Anchorage,258,229.236434
4,Europe/Paris,232,632.025862
...,...,...,...
289,America/Anguilla,1,127.000000
290,Africa/Porto-Novo,1,19.000000
291,Africa/Mbabane,1,2075.000000
292,Africa/Bujumbura,1,2582.000000


<details>
    <summary><b>Possible Solution</b></summary>

``` sql
pd.read_sql('''
    SELECT 
        airports.timezone
        ,COUNT() AS num_of_airports
    FROM 
        airports
    GROUP BY
        airports.timezone
    ORDER BY
        num_of_airports DESC
''', conn) ```
</details>

# Filtering Groups with `HAVING`

We showed that you can filter tables with `WHERE`. We can similarly filter _groups/aggregations_ using `HAVING` clauses.

## Examples of Using `HAVING`

### Simple Filtering - Number of Airports in a Country

Let's come back to the aggregation of active airlines:

In [35]:
pd.read_sql('''
    SELECT 
        COUNT() AS num,
        country
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    ORDER BY 
        num DESC
''', conn)

,num,country
0,141,United States
1,72,Russia
2,40,United Kingdom
3,37,Germany
4,34,Canada
...,...,...
190,1,Antigua and Barbuda
191,1,American Samoa
192,1,AVIANCA
193,1,ALASKA


We can see we have a lot of results. But maybe we only want to keep the countries that have more than $30$ active airlines:

In [38]:
pd.read_sql('''
    SELECT 
        country,
        COUNT() AS num
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    HAVING 
        num > 30
    ORDER BY 
        num DESC
''', conn)

,country,num
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34


In [39]:
pd.read_sql('''
    SELECT 
        country,
        COUNT() AS num
    FROM 
        airlines
    WHERE 
        active='Y' AND num > 30
    GROUP BY 
        country
    ORDER BY 
        num DESC
''', conn)

DatabaseError: Execution failed on sql '
    SELECT 
        country,
        COUNT() AS num
    FROM 
        airlines
    WHERE 
        active='Y' AND num > 30
    GROUP BY 
        country
    ORDER BY 
        num DESC
': misuse of aggregate: COUNT()

## Filtering Different Aggregations - Airport Altitudes

We can also filter on other aggregations. For example, let's say we want to investigate the `airports` table.

Specifically, we want to know the height of the _highest airport_ in a country given that it has _at least $100$ airports_.

### Looking at the `airports` Table

In [40]:
df_airports = pd.read_sql('''
    SELECT 
        *
    FROM 
        airports 
''', conn)

df_airports.head()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


### Looking at the Highest Airport

Let's first get the highest altitude for each country:

In [43]:
pd.read_sql('''
   
SELECT country, MAX(CAST(altitude as real)) AS max_altitude
FROM airports
GROUP BY country
ORDER BY country

   
''', conn)

,country,max_altitude
0,Afghanistan,7400.0
1,Albania,126.0
2,Algeria,4518.0
3,American Samoa,110.0
4,Angola,5778.0
...,...,...
235,West Bank,2485.0
236,Western Sahara,350.0
237,Yemen,7216.0
238,Zambia,4636.0


### Looking at the Number of Airports Too

We can also get the number of airports for each country.

In [44]:
pd.read_sql('''

SELECT country, MAX(CAST(altitude as real)) AS max_altitude, COUNT() AS num_airports
FROM airports
GROUP BY country
ORDER BY country

''', conn)

,country,max_altitude,num_airports
0,Afghanistan,7400.0,21
1,Albania,126.0,1
2,Algeria,4518.0,44
3,American Samoa,110.0,3
4,Angola,5778.0,26
...,...,...,...
235,West Bank,2485.0,1
236,Western Sahara,350.0,3
237,Yemen,7216.0,11
238,Zambia,4636.0,13


### Filtering on Aggregations

> Recall:
>
> We want to know the height of the _highest airport_ in a country given that it has _at least $100$ airports_.

In [47]:
pd.read_sql('''

SELECT country, MAX(CAST(altitude as real)) AS max_altitude
FROM airports
GROUP BY country
HAVING COUNT() >= 100
ORDER BY country


''', conn)

,country,max_altitude
0,Argentina,11414.0
1,Australia,4260.0
2,Brazil,8708.0
3,Canada,3939.0
4,China,14472.0
5,France,6588.0
6,Germany,2297.0
7,India,10682.0
8,Indonesia,9104.0
9,Japan,2200.0


# Joins

The biggest advantage in using a relational database (like we've been with SQL) is that you can create **joins**.

> By using **`JOIN`** in our query, we can connect different tables using their _relationships_ to other tables.
>
> Usually we use a key (*foreign key*) to tell us how the two tables are related.

There are different types of joins and each has their different use case. This is very similair to pandas `.join(), .merge()` as well as tableau relationships.

## `INNER JOIN`

> An **inner join** will join two tables together and only keep rows if the _key is in both tables_

![](images/inner_join.png)

Example of an inner join:

```sql
SELECT
    table1.column_name,
    table2.different_column_name
FROM
    table1
    INNER JOIN table2
        ON table1.shared_column_name = table2.shared_column_name
```

### Code Example for Inner Joins

Let's say we want to look at the different airplane routes

In [48]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes 
''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2
...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3
67659,67659,ZM,19016,DME,4029,FRU,2912,None,0,734
67660,67660,ZM,19016,FRU,2912,DME,4029,None,0,734
67661,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734


In [49]:
pd.read_sql('''
    SELECT 
        *
    FROM
        airlines 
''', conn)

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N
...,...,...,...,...,...,...,...,...,...
6043,6043,19828,Vuela Cuba,Vuela Cuba,6C,6CC,None,Cuba,Y
6044,6044,19830,All Australia,All Australia,88,8K8,None,Australia,Y
6045,6045,19831,Fly Europa,None,ER,RWW,None,Spain,Y
6046,6046,19834,FlyPortugal,None,PO,FPT,FlyPortugal,Portugal,Y


This is great but notice the `airline_id` column. It'd be nice to have some more information about the airlines associated with these routes.

We can do an **inner join** to get this information!

#### Inner Join Routes & Airline Data

In [50]:
pd.read_sql('''

SELECT *
FROM routes AS r
    INNER JOIN airlines AS al
        ON r.airline_id = al.id

''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,index,id,name,alias,iata,icao,callsign,country,active
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66980,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3,4171,4178,Regional Express,\N,ZL,RXA,REX,Australia,Y
66981,67659,ZM,19016,DME,4029,FRU,2912,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
66982,67660,ZM,19016,FRU,2912,DME,4029,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
66983,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y


We can also specify that we want to retain only certain columns in the `SELECT` clause:

In [51]:
pd.read_sql('''

SELECT r.airline AS r_airline, r.source AS departing, r.dest AS destination, r.stops, al.name AS airline
FROM routes AS r
    INNER JOIN airlines AS al
        ON r.airline_id = al.id
''', conn)

,r_airline,departing,destination,stops,airline
0,2B,AER,KZN,0,Aerocondor
1,2B,ASF,KZN,0,Aerocondor
2,2B,ASF,MRV,0,Aerocondor
3,2B,CEK,KZN,0,Aerocondor
4,2B,CEK,OVB,0,Aerocondor
...,...,...,...,...,...
66980,ZL,WYA,ADL,0,Regional Express
66981,ZM,DME,FRU,0,Apache Air
66982,ZM,FRU,DME,0,Apache Air
66983,ZM,FRU,OSS,0,Apache Air


#### Note: Losing Data with Inner Joins

Since data rows are kept only if _both_ tables have the key, some data can be lost

In [52]:
df_all_routes = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
''', conn)

df_routes_after_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        INNER JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

In [53]:
# Look at how the number of rows are different
df_all_routes.shape, df_routes_after_join.shape

((67663, 10), (66985, 19))

If you want to keep your data from at least one of your tables, you should use a left join instead of an inner join.

## `LEFT JOIN`

> A **left join** will join two tables together and but will keep all data from the first (left) table using the key provided.

![](images/left_join.png)

Example of a left and right join:

```sql
SELECT
    table1.column_name,
    table2.different_column_name
FROM
    table1
    LEFT JOIN table2
        ON table1.shared_column_name = table2.shared_column_name
```

### Code Example for Left Join

Recall our example using an inner join and how it lost some data since the key wasn't in both the `routes` _and_ `airlines` tables. 

In [54]:
df_all_routes = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
''', conn)

# This will lose some data (some routes not included)
df_routes_after_inner_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

# The number of rows are different
df_all_routes.shape, df_routes_after_inner_join.shape

((67663, 10), (66985, 19))

If wanted to ensure we always had every route even if the key in `airlines` was not found, we could replace our `INNER JOIN` with a `LEFT JOIN`:

In [55]:
# This will include all the data from routes
df_routes_after_left_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        LEFT JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

df_all_routes.shape, df_routes_after_left_join.shape

((67663, 10), (67663, 19))

In [56]:
df_all_routes.isna().sum()

index             0
airline           0
airline_id        0
source            0
source_id         0
dest              0
dest_id           0
codeshare     53066
stops             0
equipment        18
dtype: int64

In [57]:
df_routes_after_left_join.isna().sum()

index             0
airline           0
airline_id        0
source            0
source_id         0
dest              0
dest_id           0
codeshare     53066
stops             0
equipment        18
index           678
id              678
name            678
alias          4040
iata            787
icao            841
callsign       3860
country         678
active          678
dtype: int64

In [61]:
# This will include all the data from routes
df_routes_after_left_join = pd.read_sql('''
    SELECT 
        *
    FROM
        airlines
        LEFT JOIN routes
            ON routes.airline_id = airlines.id
''', conn)

df_all_routes.shape, df_routes_after_left_join.shape

((67663, 10), (72493, 19))

SQLlite will default to an **INNER JOIN** if not specified

## Exercise: Joins

Which airline has the most routes listed in our database?

In [65]:
# Your code here (BEST WAY USING INNER)
pd.read_sql("""

SELECT COUNT() AS num_routes, al.name, r.airline
FROM airlines AS al
    INNER JOIN routes AS r
        ON r.airline_id = al.id
GROUP BY r.airline_id
ORDER BY num_routes DESC

""", conn)

,num_routes,name,airline
0,2484,Ryanair,FR
1,2354,American Airlines,AA
2,2180,United Airlines,UA
3,1981,Delta Air Lines,DL
4,1960,US Airways,US
...,...,...,...
535,2,Papillon Grand Canyon Helicopters,HI
536,2,SmartLynx Airlines,6Y
537,2,Alaska Central Express,KO
538,2,Air Europe,PE


In [68]:
# Your code here
pd.read_sql("""

SELECT COUNT() AS num_routes, al.name, r.airline
FROM routes AS r
    LEFT JOIN airlines AS al
        ON al.id = r.airline_id
GROUP BY al.id
ORDER BY num_routes DESC

""", conn)

,num_routes,name,airline
0,2484,Ryanair,FR
1,2354,American Airlines,AA
2,2180,United Airlines,UA
3,1981,Delta Air Lines,DL
4,1960,US Airways,US
...,...,...,...
536,2,Papillon Grand Canyon Helicopters,HI
537,2,SmartLynx Airlines,6Y
538,2,Alaska Central Express,KO
539,2,Air Europe,PE


In [113]:
# Your code here
pd.read_sql("""

SELECT COUNT() AS num_routes, al.name, r.airline, r.airline_id
FROM airlines AS al
    LEFT JOIN routes AS r
        ON r.airline_id = al.id
GROUP BY r.airline_id
ORDER BY num_routes DESC

""", conn)

,num_routes,name,airline,airline_id
0,5508,Private flight,None,None
1,2484,Ryanair,FR,4296
2,2354,American Airlines,AA,24
3,2180,United Airlines,UA,5209
4,1981,Delta Air Lines,DL,2009
...,...,...,...,...
536,2,Papillon Grand Canyon Helicopters,HI,16625
537,2,SmartLynx Airlines,6Y,11814
538,2,Alaska Central Express,KO,109
539,2,Air Europe,PE,106


In [111]:
# Your code here
df = pd.read_sql("""

SELECT al.name, al.id, r.airline, r.airline_id 
FROM airlines AS al
    LEFT JOIN routes AS r
        ON al.id = r.airline_id

""", conn)

In [112]:
df

,name,id,airline,airline_id
0,Private flight,1,None,None
1,135 Airways,2,None,None
2,1Time Airline,3,None,None
3,2 Sqn No 1 Elementary Flying Training School,4,None,None
4,213 Flight Unit,5,None,None
...,...,...,...,...
72488,Vuela Cuba,19828,None,None
72489,All Australia,19830,None,None
72490,Fly Europa,19831,None,None
72491,FlyPortugal,19834,None,None


In [109]:
df['airline_id'].isna().sum()

5508

In [106]:
# Your code here
pd.read_sql("""

SELECT al.name, r.airline, r.airline_id, al.id
FROM routes AS r
    LEFT JOIN airlines AS al
        ON r.airline_id = al.id

""", conn)

,name,airline,airline_id,id
0,Aerocondor,2B,410,410
1,Aerocondor,2B,410,410
2,Aerocondor,2B,410,410
3,Aerocondor,2B,410,410
4,Aerocondor,2B,410,410
...,...,...,...,...
67658,Regional Express,ZL,4178,4178
67659,Apache Air,ZM,19016,19016
67660,Apache Air,ZM,19016,19016
67661,Apache Air,ZM,19016,19016


<details>
    <summary><b>Possible Solution</b></summary>

```sql
SELECT
    airlines.name AS airline,
    COUNT() AS number_of_routes
-- We first need to get all the relevant info via a join
FROM
    routes
    -- LEFT JOIN since we want all routes (even if airline id is unknown)
    LEFT JOIN airlines
        ON routes.airline_id = airlines.id
-- We need to group by airline's ID
GROUP BY
    airlines.id
ORDER BY
    number_of_routes DESC
```
</details>

# Level Up: Execution Order

```SQL
SELECT 
    COUNT(table2.col2) AS my_new_count
    ,table1.col2
FROM
    table1
    JOIN table2
        ON table1.col1 = table2.col2
WHERE
    table1.col1 > 0
GROUP BY
    table2.col1
```

1. `From`
2. `Where`
3. `Group By`
4. `Having`
5. `Select`
6. `Order By`
7. `Limit`

In [ ]:
pd.read_sql("""

SELECT
FROM 
WHERE 
GROUP BY
HAVING 
ORDER BY
LIMIT 

""")

In [114]:
# DOES NOT LIKE THIS AT ALL....
pd.read_sql("""

FROM routes AS r
    LEFT JOIN airlines AS al
        ON r.airline_id = al.id
GROUP BY r.airline_id
SELECT COUNT() AS num_routes, al.name, r.airline
ORDER BY num_routes DESC
LIMIT 1


""", conn)

DatabaseError: Execution failed on sql '

FROM routes AS r
    LEFT JOIN airlines AS al
        ON r.airline_id = al.id
GROUP BY r.airline_id
SELECT COUNT() AS num_routes, al.name, r.airline
ORDER BY num_routes DESC
LIMIT 1


': near "FROM": syntax error

In [115]:
df_airlines = pd.read_sql("""

SELECT * 
FROM airlines


""", conn)

In [116]:
df_routes = pd.read_sql("""

SELECT * 
FROM routes


""", conn)

In [117]:
df_routes.join(df_airlines)

ValueError: columns overlap but no suffix specified: Index(['index'], dtype='object')

In [119]:
df_routes.merge(df_airlines, left_on='airline_id', right_on='id')

,index_x,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,index_y,id,name,alias,iata,icao,callsign,country,active
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66980,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3,4171,4178,Regional Express,\N,ZL,RXA,REX,Australia,Y
66981,67659,ZM,19016,DME,4029,FRU,2912,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
66982,67660,ZM,19016,FRU,2912,DME,4029,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
66983,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
